In [5]:
import os, sys
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline
#matplotlib.use('Agg')
import matplotlib.pyplot as plt

plt.style.use('ggplot')

caffe_root = '/home/ashu/Desktop/Thesis Work/Classifier/caffe/'
print os.getcwd()
sys.path.insert(0, caffe_root + 'python')
import caffe

import argparse

from caffe.proto import caffe_pb2
import google.protobuf as pb2
import google.protobuf.text_format

import lmdb

/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/script


- Load class labels and binary state labels , save them in two seperate dictionaries

In [6]:
cls_lbl_file = caffe_root + 'data/5_class_dataset/labels/val.txt'
# read the labels file
with open(cls_lbl_file, 'r') as f:
    cls_data=f.readlines()
    
cls_data = [x.strip() for x in cls_data]

# append image name and its label in a dictionary
cls_lbl = {}
for val in cls_data:
    img, lbl = val.split(' ')
    cls_lbl[img] = lbl
    
print len(cls_data), len(cls_lbl)

# load all states indexed acc to img id in a dictionary
state_lbl = {}
st_lbl_laptop = caffe_root + 'data/5_class_dataset/labels/val_laptop_state.txt'
st_lbl_scissor = caffe_root + 'data/5_class_dataset/labels/val_scissor_state.txt'
st_lbl_suitcase = caffe_root + 'data/5_class_dataset/labels/val_suitcase_state.txt'
st_lbl_toilet = caffe_root + 'data/5_class_dataset/labels/val_toilet_state.txt'
st_lbl_umbrella = caffe_root + 'data/5_class_dataset/labels/val_umbrella_state.txt'

with open(st_lbl_laptop, 'r') as f:
    st_data_laptop=f.readlines()    
st_data_laptop = [x.strip() for x in st_data_laptop]

with open(st_lbl_scissor, 'r') as f:
    st_data_scissor=f.readlines()    
st_data_scissor = [x.strip() for x in st_data_scissor]

with open(st_lbl_suitcase, 'r') as f:
    st_data_suitcase=f.readlines()    
st_data_suitcase = [x.strip() for x in st_data_suitcase]

with open(st_lbl_toilet, 'r') as f:
    st_data_toilet=f.readlines()    
st_data_toilet = [x.strip() for x in st_data_toilet]

with open(st_lbl_umbrella, 'r') as f:
    st_data_umbrella=f.readlines()    
st_data_umbrella = [x.strip() for x in st_data_umbrella]

# concatenating all list
st_data = st_data_laptop + st_data_scissor + st_data_suitcase + st_data_toilet + st_data_umbrella

# appending the state information in one dictionary
for val in st_data:
    img, lbl = val.split(' ')
    state_lbl[img] = lbl
    
    
print len(cls_data), len(state_lbl), len(st_data)

9846 9846
9846 9846 9846


- Load model and prototxt

In [7]:
caffe.set_mode_gpu()

c_type = 'gt'
max_iter = '50000'
best_iter = '47000'
# Modify the paths given below
deploy_prototxt_file_path = caffe_root + 'models/' + c_type + '_classifier/all/test.prototxt'
# load each caffe model
caffe_model_file_path = caffe_root + 'models/gt_classifier/all/trained_model/' + \
                        'gt_all_' +  max_iter + \
                        '_iter_' + best_iter + '.caffemodel'
net = caffe.Net(deploy_prototxt_file_path, caffe_model_file_path, caffe.TEST)

- set the transformer

In [8]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

net.blobs['data'].reshape(1,        # batch size
                          3,         # 3-channel (BGR) images
                          224, 224)  # image size is 227x227

mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


- iterate through images and save fc7 and its gt label

In [9]:
global_fc7 = []
global_fc7_new = []
class_labels = []
#fc7_vectors = [[] for _ in range(5)]
#fc7_new_vectors = [[] for _ in range(5)]
#state_labels = [[] for _ in range(5)]

# extract weight
wt = net.params['cls_score'][0].data[...].clip(min=0)
img_dir = caffe_root + 'data/5_class_dataset/images/val/'

for k,v in cls_lbl.iteritems():
    img, lbl = val.split(' ')
    
    # load image, transform and feed forward
    image = caffe.io.load_image(img_dir + k)
    transformed_image = transformer.preprocess('data', image)
    net.blobs['data'].data[...] = transformed_image
    output = net.forward()
    label = int(v)
    
    # append generated fc7 to global fc7 vectors
    global_fc7.append(net.blobs['fc7'].data.copy())
    class_labels.append(int(v))
    
    if label == 0: #gt laptop
        #fc7_vectors[0].append(net.blobs['fc7'].data.copy())
        #fc7_new_vectors[0].append(net.blobs['fc7'].data.copy() * wt[0])
        #state_labels[0].append(state_lbl[k])
        global_fc7_new.append(net.blobs['fc7'].data.copy() * wt[0])
        continue
        
    if label == 1: #gt scissor
        #fc7_vectors[1].append(net.blobs['fc7'].data.copy())
        #fc7_new_vectors[1].append(net.blobs['fc7'].data.copy() * wt[1])
        #state_labels[1].append(state_lbl[k])
        global_fc7_new.append(net.blobs['fc7'].data.copy() * wt[1])
        continue
        
    if label == 2: #gt suitcase
        #fc7_vectors[2].append(net.blobs['fc7'].data.copy())
        #fc7_new_vectors[2].append(net.blobs['fc7'].data.copy() * wt[2])
        #state_labels[2].append(state_lbl[k])
        global_fc7_new.append(net.blobs['fc7'].data.copy() * wt[2])
        continue
        
    if label == 3: #gt toilet
        #fc7_vectors[3].append(net.blobs['fc7'].data.copy())
        #fc7_new_vectors[3].append(net.blobs['fc7'].data.copy() * wt[3])
        #state_labels[3].append(state_lbl[k])
        global_fc7_new.append(net.blobs['fc7'].data.copy() * wt[3])
        continue
        
    if label == 4: #gt umbrella
        #fc7_vectors[4].append(net.blobs['fc7'].data.copy())
        #fc7_new_vectors[4].append(net.blobs['fc7'].data.copy() * wt[4])
        #state_labels[4].append(state_lbl[k])
        global_fc7_new.append(net.blobs['fc7'].data.copy() * wt[4])
        continue


In [6]:
for i in range(5):
    print 'For ', i , ':'
    print 'fc7 : \n', fc7_vectors[i][:3]
    print 'fc7 new : \n', fc7_new_vectors[i][:3]
    print 'State Label : \n', state_labels[i][:3]

For  0 :
fc7 : 
[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  4.38832045]], dtype=float32), array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([[ 2.25579   ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.42316282]], dtype=float32)]
fc7 new : 
[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.00488022]], dtype=float32), array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.0004706]], dtype=float32)]
State Label : 
['0', '0', '0']
For  1 :
fc7 : 
[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.96149957]], dtype=float32), array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([[ 0.11939523,  0.        ,  2.18348169, ...,  0.        ,
         0.        ,  0.30500969]], dtype=float32)]
fc7 new : 
[array([[ 0.,  0.,  0., ...,  0.,

In [7]:
for i in range(5):
    print len(fc7_vectors[i]), len(fc7_new_vectors[i]), len(state_labels[i])

1802 1802 1802
439 439 439
2216 2216 2216
1454 1454 1454
3935 3935 3935


In [10]:
# for global fc7 only
import pickle
fc7_dir = caffe_root + 'models/' + c_type + '_classifier/all/fc7/'
if not os.path.exists(fc7_dir):
    os.makedirs(fc7_dir)
    
global_fc7_file = fc7_dir + c_type + '_fc7_global.pkl'
global_fc7_new_file = fc7_dir + c_type + '_fc7_new_global.pkl'
cls_lbl_file = fc7_dir + c_type + '_label_global.pkl'

# dump fc7 features
fileObject = open(global_fc7_file,'wb')
pickle.dump(global_fc7,fileObject)
fileObject.close()
    
# dump fc7 new features
fileObject = open(global_fc7_new_file,'wb')
pickle.dump(global_fc7_new,fileObject)
fileObject.close()

# dump state labels
fileObject = open(cls_lbl_file,'wb')
pickle.dump(class_labels,fileObject)
fileObject.close()

In [8]:
import pickle

fc7_dir = caffe_root + 'models/' + c_type + '_classifier/all/fc7/'
if not os.path.exists(fc7_dir):
    os.makedirs(fc7_dir)

objs = ['laptop', 'scissor', 'suitcase', 'toilet', 'umbrella']    

for i,o in enumerate(objs):   
    fc7_file = fc7_dir + c_type + '_fc7_' + o + '.pkl'
    fc7_new_file = fc7_dir + c_type + '_fc7_new_' + o + '.pkl'
    lbl_file = fc7_dir + c_type + '_label_' + o + '.pkl'
    
    # dump fc7 features
    fileObject = open(fc7_file,'wb')
    pickle.dump(fc7_vectors[i],fileObject)
    fileObject.close()
    
    # dump fc7 new features
    fileObject = open(fc7_new_file,'wb')
    pickle.dump(fc7_new_vectors[i],fileObject)
    fileObject.close()

    # dump state labels
    fileObject = open(lbl_file,'wb')
    pickle.dump(state_labels[i],fileObject)
    fileObject.close()
    
    print fc7_dir, fc7_file, lbl_file
    print 'done.'

print 'completed'

/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/ /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/gt_fc7_laptop.pkl /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/gt_label_laptop.pkl
done.
/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/ /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/gt_fc7_scissor.pkl /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/gt_label_scissor.pkl
done.
/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/ /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/gt_fc7_suitcase.pkl /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/gt_label_suitcase.pkl
done.
/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/ /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/all/fc7/gt_fc7_toil